In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import time
import sklearn
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from tabulate import tabulate

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.tree import DecisionTreeClassifier

evaluate={'Acc':"Accuracy", 'b_Acc':"Balanced Accuracy", 'F1':"F1 Score", 'kap':"Kappa", 'ROC':"Roc"}

In [3]:
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [4]:
def GA(train,test,cols,gen_number=25,outputcsv="GA_output.csv"):
    #defining various steps required for the genetic algorithm
    # GA adapted from https://datascienceplus.com/genetic-algorithm-in-machine-learning-using-python/
    def initilization_of_population(size,n_feat):
        population = []
        for i in range(size):
            chromosome = np.ones(n_feat,dtype=np.bool)
            chromosome[:int(0.3*n_feat)]=False
            np.random.shuffle(chromosome)
            population.append(chromosome)
        return population

    def fitness_score(population):
        scores = []
        for chromosome in population:
            logmodel.fit(X_train.iloc[:,chromosome],y_train)
            predictions = logmodel.predict(X_test.iloc[:,chromosome])
            scores.append(sklearn.metrics.f1_score(y_test,predictions,average= "macro"))
        scores, population = np.array(scores), np.array(population) 
        inds = np.argsort(scores)
        return list(scores[inds][::-1]), list(population[inds,:][::-1])

    def selection(pop_after_fit,n_parents):
        population_nextgen = []
        for i in range(n_parents):
            population_nextgen.append(pop_after_fit[i])
        return population_nextgen

    def crossover(pop_after_sel):
        population_nextgen=pop_after_sel
        for i in range(len(pop_after_sel)):
            child=pop_after_sel[i]
            child[3:7]=pop_after_sel[(i+1)%len(pop_after_sel)][3:7]
            population_nextgen.append(child)
        return population_nextgen

    def mutation(pop_after_cross,mutation_rate):
        population_nextgen = []
        for i in range(0,len(pop_after_cross)):
            chromosome = pop_after_cross[i]
            for j in range(len(chromosome)):
                if random.random() < mutation_rate:
                    chromosome[j]= not chromosome[j]
            population_nextgen.append(chromosome)
        #print(population_nextgen)
        return population_nextgen

    def generations(size,n_feat,n_parents,mutation_rate,n_gen,X_train,
                                       X_test, y_train, y_test):

        best_chromo= []
        best_score= []
        population_nextgen=initilization_of_population(size,n_feat)
        for i in range(n_gen):
            second=time.time()
            scores, pop_after_fit = fitness_score(population_nextgen)
            #print(scores[:2])
            zaman=time.time()-second

            ths.write(f"{np.mean(scores)},{np.mean(scores)},{zaman}\n")
            


            pop_after_sel = selection(pop_after_fit,n_parents)
            pop_after_cross = crossover(pop_after_sel)
            population_nextgen = mutation(pop_after_cross,mutation_rate)
            best_chromo.append(pop_after_fit[0])
            best_score.append(scores[0])
        return best_chromo,best_score
    
    



    df = pd.read_csv(train,usecols=cols)#,header=None )
    df=df.fillna(0)
    #df = df.sample(n = 10000)
    X_train =df[df.columns[0:-1]]
    #X_train=np.array(X_train)
    df[df.columns[-1]] = df[df.columns[-1]].astype('category')
    y_train=df[df.columns[-1]].cat.codes  
    df = pd.read_csv(test,usecols=cols)#,header=None )
    df=df.fillna(0)
    #df = df.sample(n = 10000)
    X_test =df[df.columns[0:-1]]
    #X_test=np.array(X_test)
    df[df.columns[-1]] = df[df.columns[-1]].astype('category')
    y_test=df[df.columns[-1]].cat.codes  

    
    ths = open(f"./{outputcsv}", "w")
    ths.write("MEAN,STD,TIME\n")
    logmodel=DecisionTreeClassifier()
    #print ('%-30s %-30s %-30s' % ("MEAN","STD","TIME"))
    chromo,score=generations(size=200,n_feat=X_train.shape[1],n_parents=120,mutation_rate=0.005,
                         n_gen=gen_number,X_train=X_train,X_test=X_test,y_train=y_train,y_test=y_test)
    #logmodel.fit(X_train.iloc[:,chromo[-1]],y_train)
    #predictions = logmodel.predict(X_test.iloc[:,chromo[-1]])
    #print("F1 Score score after genetic algorithm is= "+str(sklearn.metrics.f1_score(y_test,predictions,average= "macro")))
    ths.close()
    sonuç=[]
    for k,j in enumerate(chromo):
        temp=X_train.iloc[:,j]
        temp=list(temp.columns)
        temp.append("Label")
        sonuç.append(temp)
    
    np.save(outputcsv.replace("csv","npy"), sonuç)
    gf = pd.read_csv(outputcsv)
    gf=gf["MEAN"].values
    gf=np.argmax(gf) 
    return sonuç[gf],gf

In [5]:
def folder(f_name): #this function creates a folder named "attacks" in the program directory.
    try:
        if not os.path.exists(f_name):
            os.makedirs(f_name)
    except OSError:
        print ("The folder could not be created!")

In [6]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add

In [7]:
ml_list={"RF":RandomForestClassifier()}#,"SVC":SVC()}}

In [8]:
def target_name(name):
    df = pd.read_csv(name,usecols=["Label"])
    target_names=sorted(list(df["Label"].unique()))
    return target_names

In [9]:
folder("results")
folder("pdfs")

## Calculation of evaluations

In [10]:
def score(train_time,test_time,predict,y_test,class_based_results,repeat,cv,dname,ml,sw):
    train_time=train_time[0]
    test_time=test_time[0]
    rc=sklearn.metrics.recall_score(y_test, predict,average= "macro")
    pr=sklearn.metrics.precision_score(y_test, predict,average= "macro")
    f_1=sklearn.metrics.f1_score(y_test, predict,average= "macro")     
    accuracy=sklearn.metrics.accuracy_score(y_test, predict)
    accuracy_b=sklearn.metrics.balanced_accuracy_score( y_test,predict)
    kappa=sklearn.metrics.cohen_kappa_score(y_test, predict,labels=None, weights=None, sample_weight=None)
    try:
        roc=sklearn.metrics.roc_auc_score(y_test, predict)
    except:roc=0
    report = sklearn.metrics.classification_report(y_test, predict, target_names=target_names,output_dict=True)
    cr = pd.DataFrame(report).transpose()
    line=[dname,sw,repeat,cv,ml,accuracy,accuracy_b,pr,rc,f_1,kappa,roc,train_time,test_time]

    if class_based_results.empty:
        class_based_results =cr
    else:
        class_based_results = class_based_results.add(cr, fill_value=0)
    return class_based_results,line

In [11]:
def ML_CV(loop1,loop2,output_csv,cols,dname,sw):
    fold=5
    repetition=10

    for ii in ml_list:
        class_based_results=pd.DataFrame()#"" #pd.DataFrame(0, index=np.arange((len(target_names)+3)), columns=["f1-score","precision","recall","support"])
        cm=pd.DataFrame()
        cv=0
        lines=[["Dataset","SW","T","CV","ML","Acc","b_Acc","Prec","Rec","F1","kap","ROC","tra-T","test-T"]]
        for i in range(repetition):

            #rnd = random()
            
            #kfold = sklearn.model_selection.KFold(n_splits=fold, shuffle=True, random_state=int(rnd*100))  
            cv=0
            df = pd.read_csv(loop1,usecols=cols)#,header=None )
            df=df.fillna(0)
            X_train =df[df.columns[0:-1]]
            X_train=np.array(X_train)
            df[df.columns[-1]] = df[df.columns[-1]].astype('category')
            y_train=df[df.columns[-1]].cat.codes  


            df = pd.read_csv(loop2,usecols=cols)#,header=None )
            df=df.fillna(0)
            X_test =df[df.columns[0:-1]]
            X_test=np.array(X_test)
            df[df.columns[-1]] = df[df.columns[-1]].astype('category')
            y_test=df[df.columns[-1]].cat.codes  




            #dname=loop1  [6:-13]  
            results_y=[]
            cv+=1
            results_y.append(y_test)


            precision=[]
            recall=[]
            f1=[]
            accuracy=[]
            train_time=[]
            test_time=[]
            total_time=[]
            kappa=[]
            accuracy_b=[]

                #machine learning algorithm is applied in this section
            clf = ml_list[ii]#choose algorithm from ml_list dictionary
            second=time.time()
            clf.fit(X_train, y_train)
            train_time.append(float((time.time()-second)) )
            second=time.time()
            predict =clf.predict(X_test)
            test_time.append(float((time.time()-second)) )

            altime=0
            class_based_results,line=score(train_time,test_time,predict,y_test,class_based_results,cv,i,dname,ii,sw)
            lines.append(line)

        results = pd.DataFrame (lines[1:], columns = lines[0])
        results.to_csv(output_csv.replace("ML",ii),index=False)
        results=results.round(3)
        print (tabulate(results, headers=list(results.columns)))
        print()

In [13]:
file_list={"SYN":['SM\\DoS-SYN-6.csv', './VAL/Maze_SYN.csv'],
"ACK":['SM\\MB-ACK-4.csv', './VAL/Maze_ACK.csv'],
"BF":['SM\\MB-BF-3.csv', './VAL/Kitsune_Mir_BF.csv'],
"HTTP":['SM\\MB-HTTP-4.csv', './VAL/Maze_HTTP.csv'],
"UDP":['SM\\MB-UDP-3.csv', './VAL/BoT-IoT-UDP_DDoS.csv'],
"ARP":['SM\\MitM-ARP-4.csv', './VAL/TON_MitM.csv'],
"SCHD":['SM\\Scan-HDis-3.csv','./VAL/Scan-HDis.csv'],
"SP":['SM\\Scan-Port-3.csv', './VAL/Edge-IIoTset_Port_Scan.csv'],
"OS":['SM\\Scan-OS-3.csv','./VAL/BoT-IoT_OSScan.csv'],
"MHDis":['SM\\MB-HDis-3.csv','./VAL/MB-HDis.csv']}

In [14]:
feature_list={"SYN":['dport_sum', 'dst_port_diversity', 'IP_chksum', 'IP_DF', 'IP_flags', 'IP_id', 'IP_len', 'IP_ttl', 'pck_size', 'pck_size_mean_2', 'pck_size_mean_6', 'pck_size_mean_9', 'pck_size_mean_WE', 'pck_size_sum_of_EW', 'sport', 'sport23', 'sport_bare', 'sport_class', 'sum', 'TCP_ACK', 'TCP_ACK_ratio', 'TCP_ACK_SR', 'TCP_ACK_sum', 'TCP_flags', 'TCP_SYN_ratio', 'TCP_SYN_SR', 'TCP_SYN_sum', 'TCP_window_sum_of_EW', 'ts_sum_of_EW', 'Label'],
"ACK":['payload_bytes_mean_WE', 'payload_bytes_sum_of_EW', 'sport', 'sport_bare', 'sport_sum', 'TCP_ACK_ratio', 'TCP_ACK_SR', 'TCP_ACK_sum', 'TCP_chksum', 'TCP_seq', 'TCP_window_sum_of_EW', 'ts_mean_6', 'ts_mean_WE', 'ts_std_WE', 'ts_sum_of_EW', 'Label'],
"BF":['dport', 'dport23', 'dport_bare', 'dport_class', 'entropy_mean_WE', 'payload_bytes', 'payload_bytes_mean_2', 'pck_size_mean_WE', 'Protocol', 'TCP_ACK_ratio', 'TCP_ACK_SR', 'TCP_ACK_sum', 'TCP_PSH_ratio', 'TCP_PSH_SR', 'TCP_PSH_sum', 'TCP_RST_SR', 'TCP_window_mean_6', 'TCP_window_std_6', 'TCP_window_std_WE', 'ts_diff', 'ts_std_2', 'Label'],
"HTTP":['dport', 'dport23', 'dport_bare', 'dport_class', 'entropy_mean_6', 'entropy_mean_9', 'entropy_mean_WE', 'entropy_std_WE', 'IP_chksum', 'IP_id', 'payload_bytes_sum_of_EW', 'pck_size_sum_of_EW', 'random', 'sport_sum', 'sum', 'TCP_ACK_ratio', 'TCP_ACK_SR', 'TCP_ACK_sum', 'TCP_flags', 'TCP_PSH_SR', 'TCP_window_mean_2', 'TCP_window_mean_6', 'TCP_window_mean_9', 'TCP_window_mean_WE', 'ts', 'ts_mean_2', 'ts_std_6', 'ts_std_9', 'ts_sum_of_EW', 'Label'],
"UDP":['entropy_diff', 'entropy_std_2', 'entropy_std_6', 'entropy_std_9', 'entropy_std_WE', 'IP_chksum', 'IP_DF', 'IP_flags', 'IP_proto', 'payload_bytes_diff', 'payload_bytes_std_2', 'payload_bytes_std_6', 'payload_bytes_std_9', 'payload_bytes_std_WE', 'payload_bytes_sum_of_EW', 'pck_size_diff', 'pck_size_std_2', 'pck_size_std_6', 'pck_size_std_9', 'pck_size_std_WE', 'pck_size_sum_of_EW', 'Protocol', 'sport', 'sport23', 'sport_bare', 'sport_class', 'sport_sum', 'sum', 'TCP_ACK', 'TCP_ACK_ratio', 'TCP_ACK_SR', 'TCP_ACK_sum', 'TCP_chksum', 'TCP_dataofs', 'TCP_flags', 'TCP_options', 'TCP_PSH', 'TCP_PSH_ratio', 'TCP_PSH_SR', 'TCP_PSH_sum', 'TCP_seq', 'TCP_window', 'TCP_window_diff', 'TCP_window_mean_2', 'TCP_window_mean_6', 'TCP_window_mean_9', 'TCP_window_mean_WE', 'TCP_window_std_2', 'TCP_window_std_6', 'TCP_window_std_9', 'TCP_window_std_WE', 'TCP_window_sum_of_EW', 'ts_diff', 'ts_std_2', 'ts_std_6', 'ts_std_9', 'ts_sum_of_EW', 'UDP_chksum', 'Label'],
"ARP":['DNS_arcount', 'dport_sum', 'dst_IP_diversity', 'entropy_sum_of_EW', 'Ether_type', 'ICMP_id', 'IP_chksum', 'IP_id', 'IP_len', 'IP_ttl', 'payload_bytes', 'payload_bytes_diff', 'payload_bytes_mean_2', 'payload_bytes_mean_6', 'payload_bytes_std_2', 'payload_bytes_std_6', 'payload_bytes_std_9', 'payload_bytes_std_WE', 'payload_bytes_sum_of_EW', 'pck_size', 'pck_size_diff', 'pck_size_mean_2', 'pck_size_mean_6', 'pck_size_mean_9', 'pck_size_mean_WE', 'pck_size_std_2', 'pck_size_std_6', 'pck_size_std_9', 'pck_size_std_WE', 'pck_size_sum_of_EW', 'random', 'sport_sum', 'src_IP_diversity', 'sum', 'TCP_ACK_ratio', 'TCP_ACK_SR', 'TCP_ACK_sum', 'TCP_FIN_sum', 'TCP_PSH_ratio', 'TCP_PSH_sum', 'TCP_RST_SR', 'TCP_RST_sum', 'TCP_SYN_sum', 'TCP_window_std_6', 'TCP_window_sum_of_EW', 'ts', 'ts_diff', 'ts_mean_2', 'ts_mean_6', 'ts_mean_9', 'ts_std_2', 'ts_std_6', 'ts_std_9', 'ts_std_WE', 'ts_sum_of_EW', 'UDP_chksum', 'UDP_len', 'Label'],
"SCHD":['dport', 'dport23', 'dport_bare', 'dport_class', 'dst_port_diversity', 'entropy_mean_WE', 'entropy_std_WE', 'entropy_sum_of_EW', 'Ether_type', 'IP_add_count', 'IP_chksum', 'IP_DF', 'IP_flags', 'IP_ihl', 'IP_len', 'IP_proto', 'IP_ttl', 'IP_version', 'payload_bytes_mean_WE', 'payload_bytes_std_WE', 'payload_bytes_sum_of_EW', 'pck_size', 'pck_size_mean_2', 'pck_size_mean_6', 'pck_size_mean_9', 'pck_size_mean_WE', 'pck_size_std_WE', 'pck_size_sum_of_EW', 'Protocol', 'random', 'sport', 'sport23', 'sport_bare', 'sport_class', 'sum', 'TCP_ACK', 'TCP_ACK_ratio', 'TCP_ACK_SR', 'TCP_ACK_sum', 'TCP_chksum', 'TCP_dataofs', 'TCP_flags', 'TCP_options', 'TCP_PSH_sum', 'TCP_window_mean_WE', 'TCP_window_sum_of_EW', 'ts', 'ts_diff', 'ts_mean_2', 'ts_mean_6', 'ts_mean_9', 'ts_mean_WE', 'ts_std_2', 'ts_std_6', 'ts_std_9', 'ts_std_WE', 'ts_sum_of_EW', 'Label'],
"SP":['dst_IP_diversity', 'entropy_mean_WE', 'entropy_std_WE', 'entropy_sum_of_EW', 'IP_chksum', 'IP_DF', 'IP_flags', 'IP_id', 'payload_bytes_std_WE', 'sum', 'TCP_ACK', 'TCP_ACK_ratio', 'TCP_ACK_SR', 'TCP_ACK_sum', 'TCP_chksum', 'TCP_flags', 'TCP_seq', 'TCP_SYN', 'TCP_SYN_ratio', 'TCP_SYN_SR', 'TCP_SYN_sum', 'TCP_window_mean_2', 'TCP_window_mean_6', 'TCP_window_sum_of_EW', 'Label'],
"OS":['dport_class', 'dport_sum', 'dst_IP_diversity', 'dst_port_diversity', 'entropy', 'entropy_mean_2', 'entropy_mean_6', 'entropy_mean_9', 'entropy_mean_WE', 'entropy_std_WE', 'entropy_sum_of_EW', 'IP_id', 'IP_len', 'IP_ttl', 'payload_bytes_diff', 'payload_bytes_mean_6', 'payload_bytes_mean_9', 'payload_bytes_mean_WE', 'payload_bytes_std_2', 'payload_bytes_std_6', 'payload_bytes_std_WE', 'payload_bytes_sum_of_EW', 'pck_size', 'pck_size_mean_2', 'pck_size_mean_6', 'pck_size_mean_9', 'pck_size_mean_WE', 'pck_size_std_2', 'pck_size_std_6', 'pck_size_std_9', 'pck_size_std_WE', 'pck_size_sum_of_EW', 'Protocol', 'sport', 'sport_bare', 'sport_sum', 'sum', 'TCP_ACK', 'TCP_ACK_ratio', 'TCP_ACK_SR', 'TCP_ACK_sum', 'TCP_chksum', 'TCP_CWR', 'TCP_CWR_ratio', 'TCP_CWR_SR', 'TCP_ECE', 'TCP_ECE_ratio', 'TCP_ECE_SR', 'TCP_FIN', 'TCP_FIN_ratio', 'TCP_FIN_SR', 'TCP_FIN_sum', 'TCP_flags', 'TCP_PSH_ratio', 'TCP_PSH_SR', 'TCP_PSH_sum', 'TCP_RST_sum', 'TCP_seq', 'TCP_SYN', 'TCP_SYN_ratio', 'TCP_SYN_SR', 'TCP_SYN_sum', 'TCP_URG', 'TCP_urgptr', 'TCP_URG_ratio', 'TCP_URG_SR', 'TCP_URG_sum', 'TCP_window', 'TCP_window_mean_9', 'TCP_window_mean_WE', 'TCP_window_std_WE', 'TCP_window_sum_of_EW', 'ts_diff', 'ts_mean_2', 'ts_mean_6', 'ts_mean_9', 'ts_mean_WE', 'ts_std_2', 'ts_std_6', 'ts_std_9', 'ts_std_WE', 'ts_sum_of_EW', 'UDP_len', 'Label'],
"MHDis":['dport', 'dport23', 'dport_bare', 'dport_class', 'Ether_type', 'IP_add_count', 'IP_chksum', 'IP_DF', 'IP_flags', 'IP_ihl', 'IP_len', 'IP_proto', 'IP_ttl', 'IP_version', 'pck_size', 'pck_size_mean_2', 'pck_size_mean_6', 'pck_size_mean_9', 'pck_size_mean_WE', 'pck_size_std_WE', 'pck_size_sum_of_EW', 'Protocol', 'sport', 'sport23', 'sport_bare', 'sport_class', 'sum', 'TCP_ACK_sum', 'TCP_chksum', 'TCP_dataofs', 'TCP_flags', 'TCP_options', 'TCP_seq', 'TCP_window_mean_WE', 'TCP_window_sum_of_EW', 'ts', 'ts_diff', 'ts_mean_2', 'ts_mean_6', 'ts_mean_9', 'ts_mean_WE', 'ts_std_2', 'ts_std_6', 'ts_std_9', 'ts_std_WE', 'ts_sum_of_EW', 'Label']}

# ONLY MiTM

In [15]:
for file in file_list:
    print(file)
    features=feature_list[file]
    train=file_list[file][0]
    test=file_list[file][1]
    feature,_=GA(train,test,features,gen_number=25,outputcsv=f"{file}_RF_chosed_GA_output.csv")
    print(feature)
    output_csv=f"RF_{file}_chosed_output_ML_.csv"
    target_names=[1,0]
    ML_CV(train,test,output_csv,feature,file,5)   

SYN
['pck_size', 'IP_flags', 'IP_chksum', 'IP_DF', 'IP_ttl', 'TCP_ACK', 'sport_class', 'sport23', 'sport_bare', 'sport', 'pck_size_mean_WE', 'pck_size_sum_of_EW', 'TCP_window_sum_of_EW', 'pck_size_mean_2', 'dport_sum', 'TCP_SYN_sum', 'TCP_ACK_sum', 'TCP_ACK_ratio', 'sum', 'TCP_ACK_SR', 'pck_size_mean_6', 'pck_size_mean_9', 'Label']
    Dataset      SW    T    CV  ML      Acc    b_Acc    Prec    Rec    F1    kap    ROC    tra-T    test-T
--  ---------  ----  ---  ----  ----  -----  -------  ------  -----  ----  -----  -----  -------  --------
 0  SYN           5    1     0  RF        1        1       1      1     1      1      1    1.233     0.091
 1  SYN           5    1     1  RF        1        1       1      1     1      1      1    0.76      0.041
 2  SYN           5    1     2  RF        1        1       1      1     1      1      1    0.725     0.042
 3  SYN           5    1     3  RF        1        1       1      1     1      1      1    0.636     0.03
 4  SYN           5    1 

    Dataset      SW    T    CV  ML      Acc    b_Acc    Prec    Rec     F1     kap    ROC    tra-T    test-T
--  ---------  ----  ---  ----  ----  -----  -------  ------  -----  -----  ------  -----  -------  --------
 0  ARP           5    1     0  RF    0.694    0.506   0.543  0.506  0.444   0.017  0.506    5.048     0.139
 1  ARP           5    1     1  RF    0.675    0.493   0.47   0.493  0.437  -0.018  0.493    4.783     0.126
 2  ARP           5    1     2  RF    0.666    0.487   0.45   0.487  0.432  -0.034  0.487    4.331     0.142
 3  ARP           5    1     3  RF    0.683    0.498   0.491  0.498  0.439  -0.004  0.498    4.841     0.152
 4  ARP           5    1     4  RF    0.679    0.496   0.481  0.496  0.438  -0.011  0.496    4.774     0.175
 5  ARP           5    1     5  RF    0.608    0.445   0.389  0.445  0.405  -0.133  0.445    4.707     0.151
 6  ARP           5    1     6  RF    0.677    0.494   0.472  0.494  0.436  -0.016  0.494    4.181     0.143
 7  ARP           5